In [1]:
import io
import itertools
import networkx as nx
import nltk
import editdistance

In [3]:
def setup_envt():
    nltk.download('punkt')
    nltk.download('averaged perceptron tagger')
    print "Download complete"

In [4]:
def filter_tags(tagged, tags=['NN','JJ','NNP']):
    filtered=[]
    for item in tagged:
        if item[1] in tags:
            filtered.append(item)
    print "All tokens tagged."
    return filtered
    

In [ ]:
def normalized(tagged):
    norm=[]
    for item in tagged:
        norm.append(item[0].replace('.',''))